In [1]:
import sys
import re
import os
import numpy as np
import pandas as pd
import pysam
from Bio.Seq import Seq
from Bio import Restriction
from Bio import SeqIO
from rapidfuzz import fuzz
import rapidfuzz
import regex

In [2]:
nbp = 24
n_mismatch = 10
ref_barcode = 'A' * nbp
test_barcode = 'A' * nbp

for i in range(n_mismatch):
    test_barcode = test_barcode[:i] + 'T' + test_barcode[i+1:]
    result = fuzz.partial_ratio_alignment(test_barcode, ref_barcode)
    print(f"BP MISMATCHES: {i+1} SCORE: {result.score:.1f}")

BP MISMATCHES: 1 SCORE: 97.9
BP MISMATCHES: 2 SCORE: 95.7
BP MISMATCHES: 3 SCORE: 93.3
BP MISMATCHES: 4 SCORE: 90.9
BP MISMATCHES: 5 SCORE: 88.4
BP MISMATCHES: 6 SCORE: 85.7
BP MISMATCHES: 7 SCORE: 82.9
BP MISMATCHES: 8 SCORE: 80.0
BP MISMATCHES: 9 SCORE: 76.9
BP MISMATCHES: 10 SCORE: 73.7


In [3]:
barcode = "AAGAAAGTTGTCGGTGTCTTTGTG"
read_seq = "AAGAAAGTTGTCGGTGTCTTTGTGTGCTGAAGAAGTTGTCGGTGTCTTTGTGTTAACCTTTCTGTTGGTGCTGATATTCCGGTGCTGAAGATAGAGCGACAGGCAAGTAGGTTAACACAAAGACGCCGACAACTTTCTTCAGAAGAAAGTTGTCGGTGTCTTTGTG"

result = fuzz.partial_ratio_alignment(read_seq, barcode)
result

ScoreAlignment(score=100.0, src_start=0, src_end=24, dest_start=0, dest_end=24)

In [4]:
def find_restriction_sites(read_seq, rb):
    """A function to find all restriction sites 
    in a sequence """
    search_results = rb.search(Seq(read_seq))
    sites = list(search_results.values())[0]
    n_sites = len(sites)
    if n_sites == 0:
        sites = [-1]

    site_str = ";".join([str(x) for x in sites])
    return site_str, n_sites


def find_fuzzy(seq, tgt, min_sim):
  """
  Finds all fuzzy matches of the target sequence within the main sequence recursively.

  Args:
      seq: The main sequence to search.
      tgt: The target sequence to find fuzzy matches for.
      min_sim: Minimum similarity threshold (0-100).

  Returns:
      A list of tuples containing the matching character, matched substring, start and end positions, and similarity score.
  """
  matches = []
  best_match = None

  # Find the best alignment using a sliding window and update best_match
  for i in range(len(seq) - len(tgt) + 1):
    sub = seq[i:i + len(tgt)]
    score = fuzz.partial_ratio(tgt, sub)
    if score >= min_sim and (not best_match or score > best_match[4]):
      best_match = (seq[i], sub, i, i + len(tgt) - 1, score)

  # If a match is found, record it and continue recursively
  if best_match:
    matches.append(best_match)
    rem_seq = seq[:best_match[2]] + seq[best_match[3] + 1:]
    matches.extend(find_fuzzy(rem_seq, tgt, min_sim))

  return matches


def find_barcodes(read_seq, barcode, min_sim=92):
    """A function to find barcode sequences in reads """
    barcode_len = len(barcode)

    matches = find_fuzzy(read_seq, barcode, min_sim)
    n_matches = len(matches)
    
    if len(matches) == 0:
        pos = [-1]
        scores = [-1]
    else:
        pos = []
        scores = []
        for i, match in enumerate(matches):
            _, _, start, _, score = match
            # correct alignment positions for striped barcodes
            offset = barcode_len * i            
            start = start + offset
            pos.append(start)
            scores.append(score)

    pos_str = ";".join([str(x) for x in pos])
    score_str = ";".join([str(round(x, 2)) for x in scores])
    return pos_str, score_str, n_matches
    

def get_sequence_report(fpath, rb, barcode, barcode_rc):
    """A function to return read-level information 
    from fastq files """
    report_df = []
    
    count = -1
    stop = 1000
    for read in pysam.FastxFile(fpath):
        count += 1
        if count == stop:
            break

        # get read information
        read_seq = read.sequence
        read_seq_length = len(read_seq)
        quals = read.get_quality_array()

        # find the restrictin enzyme sites
        enzyme_pos, enzyme_matches = find_restriction_sites(read_seq, rb)

        # find forward barcodes
        fw_pos_str, fw_score_str, fw_n_matches = find_barcodes(read_seq, barcode)

        # find reverse complement barcodes
        rv_pos_str, rv_score_str, rv_n_matches = find_barcodes(read_seq, barcode_rc)

        # total BC
        total_bc_matches = fw_n_matches + rv_n_matches

        row = {
            'read_name' : read.name,
            'read_seq_length' : read_seq_length,
            'enzyme_matches' : enzyme_matches,
            'enzyme_pos' : enzyme_pos,
            'total_bc_matches' : total_bc_matches,
            'forward_bc_pos' : fw_pos_str,
            'forward_bc_score' : fw_score_str,
            'forward_bc_macthes' : fw_n_matches,
            'reverse_bc_pos' : rv_pos_str,
            'reverse_bc_score' : rv_score_str,
            'reverse_bc_macthes' : rv_n_matches,
            'mean_base_quality' : int(np.mean(quals)),
            'median_base_quality' : int(np.median(quals)),
            'min_base_quality' : np.min(quals),
            'max_base_quality' : np.max(quals),
        }
        report_df.append(row)
    return pd.DataFrame(report_df)
    

fastq_path = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/fastq/o3b01.raw.fastq"
barcode_path = "../config/barcodes.txt"
enzyme = 'NlaIII'

# set up restriction enzyme
rb = Restriction.RestrictionBatch([enzyme])

# set up get barcodes
barcode_id = os.path.basename(fastq_path).split(".")[0][2:]
code_df = pd.read_csv(barcode_path)
barcode = code_df[code_df['cell_id'] == barcode_id]['barcode'].values[0]
barcode_rc = str(Seq(barcode).reverse_complement())

df = get_sequence_report(fastq_path, rb, barcode, barcode_rc)
df.head()

,read_name,read_seq_length,enzyme_matches,enzyme_pos,total_bc_matches,forward_bc_pos,forward_bc_score,forward_bc_macthes,reverse_bc_pos,reverse_bc_score,reverse_bc_macthes,mean_base_quality,median_base_quality,min_base_quality,max_base_quality
0,6ff9aeed-b85d-415a-aa59-3ff86bbe727d,532,1,492,8,34;93;152;211;270,100.0;100.0;100.0;100.0;100.0,5,343;402;458,100.0;100.0;100.0,3,30,35,2,50
1,5faf5728-3f2e-4d37-8f62-574673a1662e,286,0,-1,4,35,100.0,1,108;167;226,100.0;100.0;100.0,3,38,42,2,50
2,4e3a9167-dac1-429e-9351-ce55485a4fae,416,0,-1,5,92;151;210,100.0;100.0;100.0,3,283;342,100.0;100.0,2,36,41,4,50
3,f536f761-c128-4cde-a247-8b3a924b3399,261,0,-1,3,38;96,100.0;97.87,2,183,95.83,1,32,37,2,50
4,23e1ac07-8810-4141-8adf-7aa4579104f4,325,0,-1,4,40;99;217;182,100.0;100.0;100.0;95.83,4,-1,-1,0,37,41,3,50


In [6]:
paths = [
     "/scratch/indikar_root/indikar1/cstansbu/scpc_test/fastq/o3b01.raw.fastq",
     "/scratch/indikar_root/indikar1/cstansbu/scpc_test/fastq/o1b04.raw.fastq",
]
barcode_path = "../config/barcodes.txt"
enzyme = 'NlaIII'

# set up restriction enzyme
rb = Restriction.RestrictionBatch([enzyme])

df = []
for fastq_path in paths:
    # set up get barcodes
    barcode_id = os.path.basename(fastq_path).split(".")[0][2:]
    code_df = pd.read_csv(barcode_path)
    barcode = code_df[code_df['cell_id'] == barcode_id]['barcode'].values[0]
    barcode_rc = str(Seq(barcode).reverse_complement())

    tmp = get_sequence_report(fastq_path, rb, barcode, barcode_rc)
    tmp['barcode_id'] = barcode_id
    df.append(tmp)

df = pd.concat(df)
df.head()

,read_name,read_seq_length,enzyme_matches,enzyme_pos,total_bc_matches,forward_bc_pos,forward_bc_score,forward_bc_macthes,reverse_bc_pos,reverse_bc_score,reverse_bc_macthes,mean_base_quality,median_base_quality,min_base_quality,max_base_quality,barcode_id
0,6ff9aeed-b85d-415a-aa59-3ff86bbe727d,532,1,492,8,34;93;152;211;270,100.0;100.0;100.0;100.0;100.0,5,343;402;458,100.0;100.0;100.0,3,30,35,2,50,b01
1,5faf5728-3f2e-4d37-8f62-574673a1662e,286,0,-1,4,35,100.0,1,108;167;226,100.0;100.0;100.0,3,38,42,2,50,b01
2,4e3a9167-dac1-429e-9351-ce55485a4fae,416,0,-1,5,92;151;210,100.0;100.0;100.0,3,283;342,100.0;100.0,2,36,41,4,50,b01
3,f536f761-c128-4cde-a247-8b3a924b3399,261,0,-1,3,38;96,100.0;97.87,2,183,95.83,1,32,37,2,50,b01
4,23e1ac07-8810-4141-8adf-7aa4579104f4,325,0,-1,4,40;99;217;182,100.0;100.0;100.0;95.83,4,-1,-1,0,37,41,3,50,b01


In [5]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
def get_first_position(position_rec):
    if ";" in position_rec:
        return int(position_rec.split(";")[0])
    else:
        return int(position_rec)

def get_last_position(position_rec):
    if ";" in position_rec:
        return int(position_rec.split(";")[-1])
    else:
        return int(position_rec)

def print_report(df):
    """A function to summarize the sequence report """
    n = len(df)
    print(f"\n###### SUMMARY SECTION ######")
    print(f"Total sequences: {len(df)}")

    print(f"\n###### SEQUENCE LENGTH ######")
    print(f"Mean sequence length: {int(df['read_seq_length'].mean())} bp")
    print(f"Median sequence length: {int(df['read_seq_length'].median())} bp")
    print(f"Min sequence length: {int(df['read_seq_length'].min())} bp")
    print(f"Max sequence length: {int(df['read_seq_length'].max())} bp")

    print(f"\n###### BASE QUALITY ######")
    print(f"Mean base quality: {int(df['mean_base_quality'].mean())}")
    print(f"Median base quality: {int(df['mean_base_quality'].median())}")
    print(f"Min base quality: {int(df['mean_base_quality'].min())}")
    print(f"Max base quality: {int(df['mean_base_quality'].max())}")

    print(f"\n###### ENZYME SECTION ######")
    n_enzymes = pd.DataFrame(df['enzyme_matches'].value_counts())
    for idx, row in n_enzymes.iterrows():
        count = row['count']
        print(f"Sequences with {idx} NlaIII sites: {count} ({100*(count / n):.2f}%)")

    print(f"\n###### FORWARD BARCODE SECTION ######")
    # get the first barcode position
    masked_df = df[df['forward_bc_pos'] != '-1']
    first_position = masked_df['forward_bc_pos'].apply(lambda x: get_first_position(x)).values
    last_position = masked_df['forward_bc_pos'].apply(lambda x: get_last_position(x)).values
    print(f"Average first barcode position on read: {int(np.mean(first_position))}")
    print(f"Average last barcode position on read: {int(np.mean(last_position))}")
    print()
    
    n_bc = pd.DataFrame(df['forward_bc_macthes'].value_counts())
    for idx, row in n_bc.iterrows():
        count = row['count']
        print(f"Sequences with {idx} barcodes: {count} ({100*(count / n):.2f}%)")


    print(f"\n###### REVERSE BARCODE SECTION ######")
    n_bc = pd.DataFrame(df['reverse_bc_macthes'].value_counts())
    for idx, row in n_bc.iterrows():
        count = row['count']
        print(f"Sequences with {idx} barcodes: {count} ({100*(count / n):.2f}%)")
    
    
    # print(df['enzyme_matches'].value_counts())


print_report(df)